In [3]:
pip install TA-Lib

  Running setup.py clean for TA-Lib
Failed to build TA-Lib
    Running setup.py install for TA-Lib: started
    Running setup.py install for TA-Lib: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\mclew\anaconda3\envs\pyvizenv\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\mclew\\AppData\\Local\\Temp\\pip-install-gfcrac3a\\ta-lib_bacc6eb7ba4240b19ad8a594301ee3da\\setup.py'"'"'; __file__='"'"'C:\\Users\\mclew\\AppData\\Local\\Temp\\pip-install-gfcrac3a\\ta-lib_bacc6eb7ba4240b19ad8a594301ee3da\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\mclew\AppData\Local\Temp\pip-wheel-5y77e1a7'
       cwd: C:\Users\mclew\AppData\Local\Temp\pip-install-gfcrac3a\ta-lib_bacc6eb7ba4240b19ad8a594301ee3da\
  Complete output (22 lines):
  C:\Users\mclew\AppData\Local\Temp\pip-install-gfcrac3a\ta-lib_bacc6eb7ba4240b19ad8a594301ee3da\setup.

In [2]:
# Import libraries
import talib as ta
from talib import abstract
from talib import MA_Type
import pandas as pd
import hvplot.pandas

ModuleNotFoundError: No module named 'talib'

In [ ]:
# Read in data from csv
df = pd.read_csv('../Data/ETF_top_8_data.csv', header=[0, 1], parse_dates=True, index_col=[0])

In [ ]:
# All availible function groups within TA-Lib
list(ta.get_function_groups())

In [ ]:
# Search for indicator by group
ta.get_function_groups()['Cycle Indicators']

In [ ]:
# Function allowing for customisable parameters in Bollinger Band
    # nbdevup = Deviation multiplier for upper band,
    # nbdevdn = Deviation multiplier for lower band,
    # Moving average type: simple moving average here
    # matype=0
def bbands_create_df(timeperiod, nbdevup, nbdevdn, ticker, matype):
    data=df[ticker]['close']
    bbands = ta.BBANDS(data, timeperiod, nbdevup, nbdevdn, matype)
    BB_df = pd.DataFrame(bbands).transpose()
    BB_df.rename(columns={0:'BB_UPPER', 1:'BB_MIDDLE', 2:'BB_LOWER'}, inplace=True)
    return BB_df

In [ ]:
# Create a list of tickers that does not contain duplicates
ticker_list = list(dict.fromkeys((df.droplevel(axis=1, level=[-1]))))

In [ ]:
# For loop that runs the Bollinger function for all tickers. It concatenates all Bollinger Band results into a single DataFrame, while also being indexed under their respective ticker
for x in ticker_list:
    # Create empty DataFrame
    empty_df = pd.DataFrame()
    # Run each Ticker through function
    bbands = bbands_create_df(timeperiod=5, nbdevup=2, nbdevdn=2, ticker=x, matype=0)
    # Create MultiIndex DataFrame containing the function results Indexed by the ticker name
    for BB_type in bbands.columns:    
        df[f'{x}', f'{BB_type}'] = bbands[BB_type]

# Re-sort index so that DataFrame is properly displayed
df = df.sort_index(axis=1)

In [ ]:
# Preview DataFrame
df

In [ ]:
# Prepare graphs for bollinger band
bb_upper = df['GDXJ']['BB_UPPER'].iloc[50:].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_middle = df['GDXJ']['BB_MIDDLE'].iloc[50:].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_lower = df['GDXJ']['BB_LOWER'].iloc[50:].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)

close = df['GDXJ']["close"].iloc[50:].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

In [ ]:
# Overlay and plot graphs
bbands_plot = close * bb_upper * bb_middle * bb_lower
bbands_plot